# CS 524 - Homework 9


## Question 1

In [17]:
min_price = [3 2 9 5 12 4]
max_price = [27 12 35 15 46 18]
returns = [0.13 0.09 0.17 0.1 0.22 0.12]
total = 80

using JuMP, Gurobi

m_abc = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))

@variable(m_abc, amt[1:6] >= 0)  # amount invested
@variable(m_abc, flag[1:6], Bin)  # flag if we are choosing that option

@objective(m_abc, Max, sum(amt[i] * returns[i] for i in 1:6))

@constraint(m_abc, sum(amt) <= total)  # ABC only has $80mil to spend
@constraint(m_abc, amt[5] <= amt[2] + amt[4] + amt[6])  # option 5 must be no more than the combined amounts of options 2, 4, 6
@constraint(m_abc, flag[3] == flag[6])  # if option 3 is chosen, then option 6 must be chosen too

for i in 1:6
    @constraint(m_abc, min_price[i]*flag[i] <= amt[i])  # set lower bound of minimum investment
    @constraint(m_abc, amt[i] <= max_price[i]*flag[i])  # set upper bound of maximum investment
end

optimize!(m_abc)

amounts = value.(amt)
flags = value.(flag)

println()
println("ABC should invest in the following options with the respective amounts (millions):")
for i in 1:6
    if flags[i] == 1
        println("    Option ", i, ": ", amounts[i])
    end
end
println()
println("This will yield a total return on investment of: \$", objective_value(m_abc), " million")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-14

ABC should invest in the following options with the respective amounts (millions):
    Option 3: 35.0
    Option 4: 5.0
    Option 5: 22.5
    Option 6: 17.5

This will yield a total return on investment of: $13.5 million
